In [1]:
#raw data - padding
'''
1. start와 end 기준으로 split 한 데이터 a를 복제해서 뒤에 붙인다
2. 길이 확인 (7초)
3. 7초 이상이면 return 
4. 7초 미만이면 1번으로 돌아간다
5. return 값을 7초(7*16000) 길이로 자른다
6. 자른 데이터 저장
'''

In [16]:
import os
import random
import pandas as pd
import numpy as np
import librosa as lb
import librosa.display as lbd

In [41]:
df = pd.read_csv('/content/drive/MyDrive/hack/csv_data/processed.csv')
df.head()

,start,end,pid,mode,filename,disease,crack_wheeze
0,1.862,5.718,160,mc,160_1b3_Al_mc_AKGC417L_0.wav,COPD,2
1,5.718,9.725,160,mc,160_1b3_Al_mc_AKGC417L_1.wav,COPD,3
2,9.725,13.614,160,mc,160_1b3_Al_mc_AKGC417L_2.wav,COPD,2
3,13.614,17.671,160,mc,160_1b3_Al_mc_AKGC417L_3.wav,COPD,2
4,17.671,19.541,160,mc,160_1b3_Al_mc_AKGC417L_4.wav,COPD,0


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6898 entries, 0 to 6897
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   start         6898 non-null   float64
 1   end           6898 non-null   float64
 2   pid           6898 non-null   int64  
 3   mode          6898 non-null   object 
 4   filename      6898 non-null   object 
 5   disease       6898 non-null   object 
 6   crack_wheeze  6898 non-null   int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 377.4+ KB


In [84]:
root = '/content/drive/MyDrive/hack/processed_audio_files_no_pad/'
test_data = '/content/drive/MyDrive/hack/processed_audio_files_no_pad/102_1b1_Ar_sc_Meditron_11.wav'

In [85]:
maxLen = 7
asdf, _ = lb.load(test_data, sr=16000)

In [86]:
testt = np.asarray(asdf)

In [87]:
testt

array([-0.0275718 , -0.03454468, -0.03094167, ...,  0.05148778,
        0.05159965,  0.05311856], dtype=float32)

In [88]:
#check length
test=[]
def check_length2(audio, maxLen):
    if len(audio)/16000 > maxLen:
        return test.append(audio)
    else:
        new = np.concatenate((audio, audio))
        return check_length2(new, maxLen)

In [89]:
check_length2(testt, maxLen=maxLen)

In [90]:
len(testt)/16000

1.7

In [93]:
len(test[0])/16000

13.6

test[0]번 지우기

In [96]:
#duplicated padding
for index, row in df.iterrows():
    filename = row['filename']
    audio_file = root + filename
    audio_data, _ = lb.load(audio_file, sr=16000)
    duplicated = np.concatenate((audio_data, audio_data))
    check_data = len(duplicated) / 16000

    if check_data > maxLen:
        test.append(duplicated)
    
    else:
        check_length2(duplicated, maxLen=maxLen)

In [98]:
len(test)

6899

In [99]:
test = np.delete(test, 0)
len(test)

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:4454: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


6898

In [100]:
np.save('/content/drive/MyDrive/hack/duplicated_audio.npy', test)

In [102]:
#slicing
sr = 16000

def audio_slicing(signal, max = 7 * sr):
    if len(signal) > max:
        return signal[:max]
    else:
        return signal

In [103]:
duplicated_7sec=[]
for i in range(len(test)):
    duplicated_7sec.append(audio_slicing(test[i]))

In [104]:
len(duplicated_7sec)

6898

In [109]:
np.save('/content/drive/MyDrive/hack/duplicated_audio_7sec.npy', duplicated_7sec)

wav 파일로 저장

In [131]:
import soundfile as sf

In [132]:
os.mkdir('/content/drive/MyDrive/hack/duplicated_7sec')

In [133]:
path = '/content/drive/MyDrive/hack/duplicated_7sec/'

In [134]:
for i in range(len(duplicated_7sec)):
    filename = 'duplicated'+'_'+str(i)+'.wav'
    sf.write(file=path+filename, data=duplicated_7sec[i], samplerate=sr)

In [135]:
from IPython.display import Audio

Audio('/content/drive/MyDrive/hack/duplicated_7sec/duplicated_0.wav')